01 Working File


In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from quantfreedom.helper_funcs import cart_product

from typing import NamedTuple
from quantfreedom.enums import CandleBodyType
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.indicators.tv_indicators import macd_tv, ema_tv

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=4500,
)

datetimes = candles[:, CandleBodyType.Timestamp].astype("datetime64[ms]")
closing_prices = candles[:, CandleBodyType.Close]
high_prices = candles[:, CandleBodyType.High]

In [3]:
macd_below = 0
fast_length = 12
slow_length = 26
signal_smoothing = 9
ema_length = 200

In [4]:
histogram, macd, signal = macd_tv(
    source=closing_prices,
    fast_length=fast_length,
    slow_length=slow_length,
    signal_smoothing=signal_smoothing,
)

In [5]:
ema = ema_tv(
    source=closing_prices,
    length=ema_length,
)

In [6]:
prev_macd = np.roll(macd, 1)
prev_macd[0] = np.nan

prev_signal = np.roll(signal, 1)
prev_signal[0] = np.nan

In [7]:
prev_macd_below_signal = prev_macd < prev_signal
current_macd_above_signal = macd > signal
price_below_ema = high_prices < ema
macd_below_bool = macd < macd_below

In [8]:
entries = (
    (price_below_ema == True)
    & (prev_macd_below_signal == True)
    & (current_macd_above_signal == True)
    & (macd_below_bool == True)
)

In [9]:
entry_signals = np.where(entries, macd, np.nan)

In [10]:
things = np.array([[10,10,10,20,20,20,30,30,30,40,40,40],[20,40,60,20,40,60,20,40,60,20,40,60]]).T
things

array([[10, 20],
       [10, 40],
       [10, 60],
       [20, 20],
       [20, 40],
       [20, 60],
       [30, 20],
       [30, 40],
       [30, 60],
       [40, 20],
       [40, 40],
       [40, 60]])

In [11]:
things[:, 0]

array([10, 10, 10, 20, 20, 20, 30, 30, 30, 40, 40, 40])

In [12]:
things[~(things[:, 0] >= things[:, 1])]

array([[10, 20],
       [10, 40],
       [10, 60],
       [20, 40],
       [20, 60],
       [30, 40],
       [30, 60],
       [40, 60]])

In [14]:
class IndicatorSettingsArrays(NamedTuple):
    ema_length: np.array
    fast_length: np.array
    macd_below: np.array
    signal_smoothing: np.array
    slow_length: np.array


cart_arrays: np.array = cart_product(
    named_tuple=IndicatorSettingsArrays(
        ema_length=np.arange(50, 201, 50),
        fast_length=np.arange(10, 41, 10),
        macd_below=np.array([0]),
        signal_smoothing=np.arange(10, 61, 10),
        slow_length=np.arange(30, 331, 50),
    )
)
indicator_settings_arrays: IndicatorSettingsArrays = IndicatorSettingsArrays(
    ema_length=cart_arrays[0].astype(np.int_),
    fast_length=cart_arrays[1].astype(np.int_),
    macd_below=cart_arrays[2].astype(np.int_),
    signal_smoothing=cart_arrays[3].astype(np.int_),
    slow_length=cart_arrays[4].astype(np.int_),
)

In [24]:
cart_arrays.T.shape

(672, 5)

In [ ]:
cart_arrays[1] < cart_arrays[4]

In [ ]:
cart_arrays.T[cart_arrays[1] < cart_arrays[4]].T

array([[50.00, 50.00, 50.00, ..., 200.00, 200.00, 200.00],
       [10.00, 10.00, 10.00, ..., 40.00, 40.00, 40.00],
       [0.00, 0.00, 0.00, ..., 0.00, 0.00, 0.00],
       [10.00, 10.00, 10.00, ..., 60.00, 60.00, 60.00],
       [30.00, 80.00, 130.00, ..., 230.00, 280.00, 330.00]])

In [ ]:
fig = go.Figure()
datetimes = candles[:, CandleBodyType.Timestamp].astype("datetime64[ms]")
fig = make_subplots(
    cols=1,
    rows=2,
    shared_xaxes=True,
    subplot_titles=["Candles", "MACD"],
    row_heights=[0.6, 0.4],
    vertical_spacing=0.1,
)
# Candlestick chart for pricing
fig.append_trace(
    go.Candlestick(
        x=datetimes,
        open=candles[:, CandleBodyType.Open],
        high=candles[:, CandleBodyType.High],
        low=candles[:, CandleBodyType.Low],
        close=candles[:, CandleBodyType.Close],
        name="Candles",
    ),
    col=1,
    row=1,
)
fig.append_trace(
    go.Scatter(
        x=datetimes,
        y=ema,
        name="EMA",
        line_color="yellow",
    ),
    col=1,
    row=1,
)
ind_shift = np.roll(histogram, 1)
ind_shift[0] = np.nan
colors = np.where(
    histogram >= 0,
    np.where(ind_shift < histogram, "#26A69A", "#B2DFDB"),
    np.where(ind_shift < histogram, "#FFCDD2", "#FF5252"),
)
fig.append_trace(
    go.Bar(
        x=datetimes,
        y=histogram,
        name="histogram",
        marker_color=colors,
    ),
    row=2,
    col=1,
)
fig.append_trace(
    go.Scatter(
        x=datetimes,
        y=macd,
        name="macd",
        line_color="#2962FF",
    ),
    row=2,
    col=1,
)
fig.append_trace(
    go.Scatter(
        x=datetimes,
        y=signal,
        name="signal",
        line_color="#FF6D00",
    ),
    row=2,
    col=1,
)
fig.append_trace(
    go.Scatter(
        x=datetimes,
        y=entry_signals,
        mode="markers",
        name="entries",
        marker=dict(
            size=12,
            symbol="circle",
            color="#00F6FF",
            line=dict(
                width=1,
                color="DarkSlateGrey",
            ),
        ),
    ),
    row=2,
    col=1,
)
# Update options and show plot
fig.update_layout(height=800, xaxis_rangeslider_visible=False)
fig.show()